# Implementação do agente ReAct proposto pelo vídeo

## Lógica do ReAct

1.   Pensar: Compreender o que deve ser feito
2.   Agir: Escolher a melhor ação/tool e executar
3.   PAUSE: para aguardar o resultado
4.   Observação: Recebe o resultado

*   Caso a resposta ainda não foi fornecida, retornar para o "Pensar"

## Instalação do modelo Groq

In [2]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 1.8 MB/s eta 0:00:00


## Configuração da API key do groq

In [3]:
import os
os.environ['GROQ_API_KEY'] = "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"

## Fazendo conexão da Api com o Groq

In [4]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in the field of natural language processing (NLP) due to their numerous benefits and applications. The importance of fast language models can be understood from the following perspectives:

1. **Improved User Experience**: Fast language models enable real-time processing and response generation, providing users with instant feedback and results. This is particularly important for applications such as chatbots, virtual assistants, and language translation systems, where users expect quick and accurate responses.
2. **Increased Efficiency**: Fast language models can process large volumes of text data quickly, making them ideal for tasks such as text classification, sentiment analysis, and entity extraction. This efficiency enables businesses and organizations to analyze and understand large amounts of text data, leading to better decision-making and improved operations.
3. **Real-time Applications**: Fast language models are essential for real-time applic

## Construção da estrutura da Classe do Agente

In [8]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self):
    completion = client.chat.completions.create(
        messages=self.messages,
        model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content

## Escrita do System Prompt

In [5]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

## Criação das Tool/Ferramentas a serem usadas

In [6]:
# tools
def calculate(operation):
  return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

## Execução manual do Agente

In [9]:
neil_tyson = Agent(client, system_prompt)

In [10]:
result = neil_tyson("What is the mass of Earth times 5?")
print(result)

Thought: To find the mass of Earth times 5, I first need to find the mass of Earth.
Action: get_planet_mass: Earth
PAUSE


In [11]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [12]:
result = neil_tyson()
print(result)

In [14]:
observation =  get_planet_mass("Earth")
print(observation)

5.972e+24


In [15]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: Now that I have the mass of Earth, I can multiply it by 5 to find the result.
Action: calculate: 5.972e+24 * 5
PAUSE


In [16]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [17]:

result = neil_tyson()
print(result)

In [19]:
observation = calculate("3.285e23 * 5")
print(observation)

1.6425e+24


In [23]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: The calculation is complete, and I have the result of the mass of Earth times 5, but it seems I made an error in my previous thought. The correct result is 2.986e+25, no, the observation says 1.6425e+24 is incorrect it should be  5.972e+24 * 5 = 2.986e+25, however the result of the calculation is actually 1.986e+25 * 5 = 9.931e+24 5.972e+24 * 5 = 2.986e+25, no 5.972e+24 * 5 = 2.986e+25. 
5.972e+24 * 5 = 2.986e+25. I made another mistake,  5.972e+24 * 5 = 2.986e+25, The observation is actually 1.9860e+25 * 0.5 = 9.93e+24, no. 
The correct result of 5.972e+24 * 5 = 2.986e+25. 
Answer: The mass of Earth times 5 is 2.986e+25


In [24]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

## Construção do Loop para execução automática do Agente

In [32]:
import re

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt)
  tools = ["calculate", "get_planet_mass"]
  next_prompt = query
  i=0
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result and "Action" in result:
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
      choise_tools = action[0][0]
      arg = action[0][1]

      if choise_tools in tools:
        result_tool = eval(f"{choise_tools}('{arg}')")
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = "Observation: Tool not found"

      print(next_prompt)
      continue

    if "Answer" in result:
      break

agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")

Thought: To solve this problem, I need to find the masses of Earth and Saturn, add them together, and then multiply the result by 2. I'll start by finding the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Saturn.

Action: get_planet_mass: Saturn
PAUSE
Observation: 5.683e+26
Thought: I now have the masses of both Earth and Saturn. The next step is to add these two masses together.

Action: calculate: 5.972e+24 + 5.683e+26
PAUSE
Observation: 5.74272e+26
Thought: Now that I have the sum of the masses of Earth and Saturn, I need to multiply this result by 2 to get the final answer.

Action: calculate: 5.74272e+26 * 2
PAUSE
Observation: 1.148544e+27
Thought: I have now completed all the necessary calculations. The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.

Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e+27.
